In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import route_dynamics.route_elevation.base_df as base
import geopandas as gpd

from shapely.geometry import mapping

In [3]:
routes_shp = '../data/rt' + str(101) + '_pts2.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

In [4]:
dat1 = pd.read_csv("../data/Trip183.csv", low_memory=False)
dat2 = pd.read_csv("../data/Zon183Unsum.csv")

In [5]:
trip183 = dat1[['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats',
                'Period', 'AnnRides']]

In [6]:
trip183unsum = dat2[['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs']]

In [7]:
period = 'AM'
route = 101
df = trip183unsum
df = df.drop(df[(df.Period != period)].index)
#df = df.drop(df[(df.InOut != direction)].index)
df = df.drop(df[(df.Route != route)].index)
df

,Route,Dir,Trip_ID,InOut,STOP_SEQ,STOP_ID,Period,AveOn,AveOff,AveLd,Obs
200767,101,N,39170951,I,18.0,532,AM,0.5,5.7,35.6,38
200771,101,N,39170923,I,18.0,532,AM,0.5,7.8,21.9,34
200774,101,N,39170913,I,18.0,532,AM,0.4,8.5,26.5,40
200782,101,N,39171034,I,18.0,532,AM,0.8,10.2,29.5,49
200789,101,N,39170869,I,18.0,532,AM,1.5,11.5,37.7,33
...,...,...,...,...,...,...,...,...,...,...,...
203288,101,S,39170966,O,6.0,99271,AM,0.2,0.6,16.4,38
203289,101,S,39171018,O,6.0,99271,AM,0.1,0.4,12.4,35
203299,101,S,39171006,O,6.0,99271,AM,0.3,0.7,15.7,31
203304,101,S,39170987,O,6.0,99271,AM,0.2,0.4,14.0,36


In [8]:
final_df = df.sort_values(by=['InOut','Trip_ID', 'STOP_SEQ', 'STOP_ID'])
final_df

,Route,Dir,Trip_ID,InOut,STOP_SEQ,STOP_ID,Period,AveOn,AveOff,AveLd,Obs
201362,101,N,39170866,I,1.0,55938,AM,9.6,0.0,9.6,34
201422,101,N,39170866,I,2.0,55939,AM,1.4,0.0,10.7,35
201206,101,N,39170866,I,3.0,46466,AM,0.7,0.0,11.4,35
201228,101,N,39170866,I,4.0,46468,AM,0.3,0.0,11.7,35
201307,101,N,39170866,I,5.0,46472,AM,1.4,0.0,13.1,35
...,...,...,...,...,...,...,...,...,...,...,...
202446,101,S,39171277,O,17.0,45303,AM,0.1,4.8,3.2,26
202173,101,S,39171278,O,1.0,1080,AM,1.5,0.0,1.5,38
202232,101,S,39171278,O,2.0,1108,AM,3.6,0.0,5.1,38
202006,101,S,39171278,O,3.0,455,AM,1.9,0.1,6.9,38


In [9]:
final_df = final_df[['InOut','STOP_SEQ', 'STOP_ID', 'AveLd']]
final_df = final_df.drop_duplicates(subset=['InOut','STOP_SEQ'], keep='first')
len(final_df)

42

In [10]:
final_df = final_df.sort_values(by=['InOut','STOP_SEQ'])
final_df.reset_index()

,index,InOut,STOP_SEQ,STOP_ID,AveLd
0,201362,I,1.0,55938,9.6
1,201422,I,2.0,55939,10.7
2,201206,I,3.0,46466,11.4
3,201228,I,4.0,46468,11.7
4,201307,I,5.0,46472,13.1
5,201102,I,6.0,45301,28.4
6,201140,I,7.0,45320,29.5
7,201480,I,8.0,79520,36.3
8,201515,I,9.0,79530,40.4
9,201582,I,10.0,79540,40.5


In [11]:
final_df['AveLd'].mean()

23.599999999999998

In [12]:
empty_bus = 14000

In [13]:
final_df['Pass_Mass'] = final_df['AveLd']*80
final_df['Total_Mass'] = final_df['Pass_Mass'] + empty_bus
#final_df = final_df[['InOut', 'STOP_SEQ', 'STOP_ID', 'AveLd', 'Pass_Load', 'Pass_Mass', 'Total_Mass', 'geometry']]
final_df

,InOut,STOP_SEQ,STOP_ID,AveLd,Pass_Mass,Total_Mass
201362,I,1.0,55938,9.6,768.0,14768.0
201422,I,2.0,55939,10.7,856.0,14856.0
201206,I,3.0,46466,11.4,912.0,14912.0
201228,I,4.0,46468,11.7,936.0,14936.0
201307,I,5.0,46472,13.1,1048.0,15048.0
201102,I,6.0,45301,28.4,2272.0,16272.0
201140,I,7.0,45320,29.5,2360.0,16360.0
201480,I,8.0,79520,36.3,2904.0,16904.0
201515,I,9.0,79530,40.4,3232.0,17232.0
201582,I,10.0,79540,40.5,3240.0,17240.0


In [14]:
stop_ids = final_df['STOP_ID'].values
stop_ids

array([55938, 55939, 46466, 46468, 46472, 45301, 45320, 79520, 79530,
       79540, 79555, 79560, 99252, 99254, 99264, 99267,   621,   532,
         565,  1121,  1075,  1080,  1108,   455,   501,   623, 99271,
       99266, 99263, 99253, 79570, 79580, 79590, 79600, 79610, 46440,
       46450, 45303, 45285, 45288, 59841, 59839])

In [15]:
stops_riders = pd.DataFrame(stop_ids, columns = ['STOP_ID'])
stops_riders.head()

,STOP_ID
0,55938
1,55939
2,46466
3,46468
4,46472


In [16]:
len(stops_riders)

42

In [17]:
stops = gpd.read_file(stops_shp)
stops.head() 

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
0,1,142,0,YES,None,80761,225897,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-earth,None,0,8414.0,POINT (-122.22345 47.46961)
1,2,142,0,YES,None,80762,312598,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Base plate,None,0,8415.0,POINT (-122.22346 47.46579)
2,3,142,0,YES,None,80763,226757,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Earth,Single,0,8416.0,POINT (-122.22378 47.44596)
3,4,142,0,NO,None,80764,312386,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,8417.0,POINT (-122.22404 47.44358)
4,5,142,0,YES,None,80765,226757,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Earth,None,0,8418.0,POINT (-122.22358 47.44595)


In [18]:
stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)

In [19]:
stops['ROUTE_LIST']

0        F 153
1          153
2          153
3          153
4          153
         ...  
28565      3 4
28566      3 4
28567      3 4
28568        0
28569        0
Name: ROUTE_LIST, Length: 28570, dtype: object

In [20]:
stops_list = pd.DataFrame(columns=stops.columns)
stops_list

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry


In [21]:
test = pd.DataFrame(stops.loc[1])

In [22]:
test.transpose()

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
1,2,142,0,YES,None,80762,312598,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Base plate,None,0,8415.0,POINT (-122.22345991745925 47.465790007896665)


In [23]:
for i in range(0, len(stops)):

    for j in range(len(stops_riders)):
        if stops_riders['STOP_ID'][j]==stops['STOP_ID'][i]:
            row = pd.DataFrame(stops.iloc[i])
            stops_list = pd.concat([stops_list,row.transpose()])
        else:
             pass
    

In [24]:
stops_list

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
1773,2774,141,2,YES,None,623,173116,ACT,TUN,MET,...,Intl District Station AcRd,Tunnel Rts,KCM,None,None,Concrete,None,0,68.0,POINT (-122.32821306920877 47.59766264472039)
1863,2864,141,2,YES,Unknown,1075,91786,CLO,REG,SEA,...,9th Ave,B1 <=12 rts,KCM,Unknown,Unknown,Unknown,Unknown,0,100012289.0,POINT (-122.33222314698305 47.61359799076774)
1865,2866,141,2,YES,Unknown,1080,57697,CLO,REG,SEA,...,9th Ave,B1 <=12 rts,KCM,Unknown,Unknown,Unknown,Unknown,0,100000500.0,POINT (-122.33245275248797 47.614107722005436)
1955,2956,141,2,YES,None,565,167086,ACT,TUN,MET,...,University St Sta AcRd,Tunnel Rts,KCM,None,None,Concrete,None,0,56.0,POINT (-122.33616423925251 47.608247620160846)
4104,3105,141,2,YES,None,1108,106225,ACT,TUN,MET,...,Westlake Sta AcRd,Tunnel Rts,KCM,None,None,Concrete,None,0,118.0,POINT (-122.33750393664289 47.61139091076956)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25217,24218,142,0,YES,None,46450,312566,ACT,REG,RNT,...,S 3rd St,A2 <=6 rts,KCM,Away,Light,Band,None,0,3831.0,POINT (-122.21295382349285 47.4796637998879)
26263,26264,142,0,YES,None,1108,106225,ACT,TUN,MET,...,Westlake Sta AcRd,Tunnel Rts,KCM,None,None,Concrete,None,0,118.0,POINT (-122.33750393664289 47.61139091076956)
28381,28382,142,0,YES,None,79520,312450,ACT,REG,RNT,...,SW Sunset Blvd,A2 <=6 rts,KCM,Toward,2in metal,Cncrt-earth,None,1,8247.0,POINT (-122.21786409952078 47.479823365113)
28384,28385,142,0,YES,Unknown,79530,308969,ACT,REG,RNT,...,SW Sunset Blvd,A1 <=2 rts,KCM,Toward,2in metal,Base plate,Single,1,8250.0,POINT (-122.22455829140581 47.47786794534816)


In [25]:
coord_list_full = stops_list.drop_duplicates('STOP_ID')
coord_list = coord_list_full[['STOP_ID', 'geometry']]
len(coord_list)

42

In [ ]:
coord_list.loc[coord_list['STOP_ID'] == 22390].geometry.values

In [ ]:
filler = np.zeros(len(final_df))

In [ ]:
final_df = final_df.assign(geometry = filler)

In [ ]:
for i in range(len(final_df)):
    stopid = final_df['STOP_ID'].iloc[i]
    copy = coord_list.loc[coord_list['STOP_ID'] == stopid].geometry.values
    final_df['geometry'].iloc[i] = copy

In [ ]:
final_df.head(20)

In [ ]:
len(final_df)

In [ ]:
geometry = final_df.geometry.values

In [ ]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i][0])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
xy_df['STOP_ID'] = stops_list['STOP_ID'].values

In [ ]:
xy_df

In [ ]:
stops_riders.value_counts()

In [ ]:
pd.merge(stops_riders, xy_df, on = 'STOP_ID', how='')

In [ ]:
def route_ridership(period, direction, route):
    
    df = trip183unsum
    df = df.drop(df[(df.Period != period)].index)
    df = df.drop(df[(df.InOut != direction)].index)
    df = df.drop(df[(df.Route != route)].index)

    final_df = df.sort_values(by=['Trip_ID', 'STOP_SEQ', 'STOP_ID'])
    seq_id = final_df[['STOP_SEQ', 'STOP_ID']]
    seq_id2 = seq_id.drop_duplicates(subset=[,'STOP_SEQ'], keep='first')
    seq_id3 = seq_id2.sort_values(by='STOP_SEQ')
    stopid_dic = dict(zip(seq_id3.STOP_SEQ, seq_id3.STOP_ID))
    riders = final_df.pivot(index='STOP_SEQ', columns='Trip_ID', values='AveLd')

    keyfind = list(riders.columns)
    mass_bus = [trip_dict[x] for x in keyfind]
    mode = mode_mass = max(set(mass_bus), key=mass_bus.count)
    rider_columns = list(riders.columns)
    kept_columns = []

    for i in range(0, len(mass_bus)):
        if mass_bus[i] == mode:
            kept_columns.append(rider_columns[i])

    riders_interm = riders[kept_columns]
    riders_kept = pd.DataFrame((riders_interm.mean(axis=1)), columns=['Mean'])
    riders_kept.Mean*=80
    riders_kept.reset_index(inplace=True)
    riders_kept.replace({"STOP_SEQ": stopid_dic}, inplace=True)
    riders_kept.columns = ['STOP_ID', 'Mean']



    return final_df, riders_kept, mode_mass

In [ ]:
df_45, riders_45, mass_45 = route_ridership('AM', 'I', 45)

In [ ]:
riders_45

In [ ]:
def stop_coord(num, riders_num):
    route_num = num
    
    route = base.create_gdf(routes_shp, 6)
    #points = base.extract_point_df(route)
    stops = gpd.read_file(stops_shp)
    stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)
    
    stops_list = pd.DataFrame()
    for i in range(0, len(stops)):

        if str(22) in (stops['ROUTE_LIST'][i]):
            for x in stops['ROUTE_LIST'][i].split(' '):
                if str(22) == x:
                    row = pd.DataFrame(stops.iloc[i])
                    stops_list = pd.concat([stops_list,row.transpose()])
                else:
                    pass
        else:
            pass
        
    stop_ids = stops_list['STOP_ID'].values
    
    geometry = stops_list.geometry.values

    xy = []
    for i in range(len(geometry)):
        dic = mapping(geometry[i])
        coords = dic['coordinates']
        xy.append(coords)
        xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
        xy_df['coordinates'] = xy
    
    xy_df['STOP_ID'] = stops_list['STOP_ID'].values
    
    df = pd.DataFrame(riders_num, columns = ['STOP_ID'])
    #df_ind = df.reset_index()
    #df_ind.columns = ['STOP_SEQ', 'STOP_ID', 'Mean']
    
    df_comb = xy_df.merge(df, how='right', on="STOP_ID")
    df_comb = df_comb.drop_duplicates(subset=['STOP_ID'], keep='first')
    df_comb = df_comb[['STOP_ID', 'coordinates']]
    #df_combine = df_comb.sort_values(by='STOP_SEQ')
    
    return xy_df, df_comb

In [ ]:
stop_coord(22, stop_ids)

In [ ]:
coordinates_45, final_45 = stop_coord(45, riders_45)

In [ ]:
coordinates_45.head()

In [ ]:
final_45.head()

In [ ]:
df_45_ami, riders_45ami, mass_45ami = route_ridership('AM', 'I', 45)
df_45amo, riders_45amo, mass_45amo = route_ridership('AM', 'O', 45)

In [ ]:
df_45pmi, riders_45pmi, mass_45pmi = route_ridership('PM', 'I', 45)
df_45pmo, riders_45pmo, mass_45pmo = route_ridership('PM', 'O', 45)

In [ ]:
riders_45pmi.head()

In [ ]:
df_45ami, riders_45ami, mass_45ami = route_ridership('AM', 'I', 45)
df_45midi, riders_45midi, mass_45midi = route_ridership('MID', 'I', 45)
df_45pmi, riders_45pmi, mass_45pmi = route_ridership('PM', 'I', 45)
df_45xnti, riders_45xnti, mass_45xnti = route_ridership('XNT', 'I', 45)
df_45xevi, riders_45xevi, mass_45xevi = route_ridership('XEV', 'I', 45)

In [ ]:
riders45ami = riders_45ami / 80
riders45midi = riders_45midi / 80
riders45pmi = riders_45pmi / 80
riders45xevi = riders_45xevi / 80
riders45xnti = riders_45xnti / 80

In [ ]:
riders45pmo = riders_45pmo / 80
riders45pmi = riders_45pmi / 80

In [ ]:
riders_45ami.head()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(riders45pmo['Mean'], label='PM Outbound Segment', linewidth=4)
plt.plot(riders45pmi['Mean'], label='PM Inbound Segment', linewidth=4)
plt.legend()
plt.xlabel('Stop Sequence', size=16)
plt.ylabel('Average Number of Riders', size=16)
plt.title('Route 45 Inbound vs Outbound Riders', size=24)
plt.savefig('Outbound_Inbound_45.png', dpi=300)

In [ ]:
plt.figure(figsize=(8,8))
AM_plot = plt.plot(riders45ami['Mean'], label='Morning (5AM-9AM)', linewidth=4)
MID_plot = plt.plot(riders45midi['Mean'], label='Mid-Day (9AM-3PM)', linewidth=4)
PM_plot = plt.plot(riders45pmi['Mean'], label='Late Afternoon (3PM-7PM)', linewidth=4)
XEV_plot = plt.plot(riders45xevi['Mean'], label='Evening (7PM-10PM)', linewidth=4)
XNT_plot = plt.plot(riders45xnti['Mean'], label='Late Night (10PM-5AM)', linewidth=4)
plt.legend()
plt.xlabel('Stop Sequence', size=16)
plt.ylabel('Average Number of Riders', size=16)
plt.title('Number of Riders on Route 45 (Inbound)', size=24)
#plt.savefig('Ridership_daysegments.png', dpi=300)

In [ ]:
df_45amo, riders_45amo, mass_45amo = route_ridership('AM', 'O', 45)
df_45mido, riders_45mido, mass_45mido = route_ridership('MID', 'O', 45)
df_45pmo, riders_45pmo, mass_45pmo = route_ridership('PM', 'O', 45)
df_45xevo, riders_45xevo, mass_45xevo = route_ridership('XEV', 'O', 45)
df_45xnto, riders_45xnto, mass_45xnto = route_ridership('XNT', 'O', 45)

In [ ]:
AM_ploto = plt.plot(riders_45amo['Mean'], label='AM')
MID_ploto = plt.plot(riders_45mido['Mean'], label='MID')
PM_ploto = plt.plot(riders_45pmo['Mean'], label='PM')
XEV_ploto = plt.plot(riders_45xevo['Mean'], label='XEV')
XNT_ploto = plt.plot(riders_45xnto['Mean'], label='XNT')
plt.legend()